### **RAG with PDF**

In [11]:
from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import getpass
import os
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
# Load the API key from the environment
def get_mistral_api_key():
    key = os.getenv("MISTRAL_API_KEY")
    if not key:
        key = getpass.getpass("Enter your Mistral API key: ")
        os.environ["MISTRAL_API_KEY"] = key
    return key

# Get the Mistral API key
mistral_api_key = get_mistral_api_key()

In [13]:
loader = PyMuPDFLoader("/Users/Shatten/Edu/Python/MLOps/p_eBook.pdf")

In [14]:
pages = loader.load_and_split()

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

In [16]:
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
)

/Users/Shatten/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [17]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [18]:
retriever = vectorstore.as_retriever()

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
llm = ChatMistralAI(model="mistral-small-2501", temperature=0, max_retries=2)

In [21]:
template = """SYSTEM: You are a question answer bot.
                Be factual in your response.
                Respond to the following question: {question} only from 
                the below context : {context}.
                If you don't know the answer, just say that you don't know.
          """
prompt = PromptTemplate.from_template(template)

In [22]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser ()
)

In [23]:
chain.invoke("What is the key steps of career growth?")

'Based on the provided context, the key steps of career growth related to "meaningful projects" are:\n\n1. **Working on projects**: This step involves deepening your skills, building a portfolio, and creating impact.'

### **RAG with webpages**

In [25]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [26]:
URL = "https://www.theverge.com/2024/4/18/24133808/meta-ai-assistant-llama-3-chatgpt-openai-rival"

In [27]:
loader = WebBaseLoader(URL)

In [28]:
chain.invoke("Hove many model Meta introduced?")

HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}